In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/oracle/ClosedStack_data_Virtual.csv', low_memory= False, encoding='latin-1')

In [ ]:
df_new = df[[ 'Job ID','Hardware ID', 'Hardware Name', 'Solution ID', 'Ticket Site','Site Name', 'Job Created']]
df_filtered1 = df_new.dropna(subset=['Job ID', 'Job Created', 'Solution ID']).copy()
df_filtered1.loc[:, 'Job Created'] = pd.to_datetime(df_filtered1['Job Created']).dt.strftime('%Y-%m-%d')

In [ ]:
"Job ID","Job Status","Job Klass","Job Hardware ID","Job Operation","Job Created",
"Job Updated","Job Duration in Seconds","Job Ticket ID","Hardware ID","Hardware Name",
"Hardware Image ID","Hardware Solution ID","Hardware Site ID","Image ID","Image Name","Site ID",
"Site Name","Ticket Site Name","First Site Name","Solution ID","Solution Name","Ticket ID","Ticket Site"
,"Hardware Compute Config ID","Compute Config ID","Compute Config Name","Compute Config CPU","Compute Config Memory"
,"Compute Config Disk","Compute Config SKU"

#first site name, job created


In [ ]:
df_filtered1

,Job ID,Hardware ID,Hardware Name,Solution ID,Ticket Site,Site Name,Job Created
0,862ffa76-9516-451d-bd2c-d7881ad62be1,0db7b4d8-6cfc-4a88-8896-a3b3b442f0ce,cerncomgucproj01,03145e08-3c9f-4514-9acc-02a1657fee80,a3920a41-5eba-4635-a535-82462555337e,NaN,2022-12-15
1,2c6fb3c3-673a-455e-ba56-29165c387ff7,421ec789-0ece-4e36-a61d-c9492214339a,jamesieolydb01,03145e08-3c9f-4514-9acc-02a1657fee80,db1c6643-54fd-4d71-ad9c-9f5bd6baf857,STCK-25-DC01,2022-12-15
2,a36e2657-b11d-4410-bef2-de8ef9ce9cc0,96223747-c5ef-469d-a15e-c57fa8bc6538,eastmeaxlic05,71e4908b-9e81-4641-929c-08dd9876a69b,a5bd8b00-e21c-47fa-826c-8aea3ac9f9ea,NaN,2022-12-15
3,12749217-adc6-4bb7-a905-4feed1a6a15e,96223747-c5ef-469d-a15e-c57fa8bc6538,eastmeaxlic05,71e4908b-9e81-4641-929c-08dd9876a69b,a5bd8b00-e21c-47fa-826c-8aea3ac9f9ea,NaN,2022-12-15
4,412c3b6e-6183-45e8-88c3-2768604dfbe7,eff3ec9b-19df-43f5-be16-3fadd12c8728,eastmeaxws18,9aa697ea-ca66-4064-b669-096c647d26a4,b32bd8cb-9709-47ca-93f2-4eab4c26546f,CTC-KC-I,2022-12-15
...,...,...,...,...,...,...,...
88637,b028e725-0b5f-4417-8d7b-c4d623537242,e103c23b-bcdf-46d5-a035-cc75c5a76a14,CTRSNYCRWL06,129e6f40-4708-4d9d-93a7-99f980eaf345,a5bd8b00-e21c-47fa-826c-8aea3ac9f9ea,CTC-LS-IV,2018-11-29
88638,be578f38-6055-48b5-a82f-d8c9ceafc255,4770550a-40bc-4304-bc60-674874ecb87c,COMMFLCRWL06,129e6f40-4708-4d9d-93a7-99f980eaf345,a3920a41-5eba-4635-a535-82462555337e,CTC-KC-III,2018-11-29
88639,76776428-15c9-44a2-9d7a-86da8d5ea919,ba9dda22-5cbc-406b-b5cc-65ee9424a284,cerncomisync02,72d96cb0-9e0b-4851-b42c-60e8705c6d72,3d80e882-9f27-4211-a7c5-9ef938661869,NaN,2018-11-29
88640,f077e233-a555-4547-919a-f2e99f31eabb,b5a23e69-a520-4abf-8fa1-c6922550937a,cerncomisync03,72d96cb0-9e0b-4851-b42c-60e8705c6d72,3d80e882-9f27-4211-a7c5-9ef938661869,NaN,2018-11-29


In [ ]:
def df_to_X_y(df_filtered1, window_size=5):

  X = []
  y = []

  for i in range(len(df_filtered1)-window_size):
    row = [[a] for a in df_filtered1[i:i+window_size]]
    X.append(row)
    label = df_filtered1[i+window_size]
    y.append(label)

  return np.array(X), np.array(y)

In [ ]:
df_filtered1['Solution ID']
solution_id_encoded = df_filtered1['Solution ID'].astype('category').cat.codes
solution_id_encoded
#string type, not case sensetive/space

0          2
1          2
2         75
3         75
4        102
        ... 
88637     11
88638     11
88639     77
88640     77
88641     77
Length: 57579, dtype: int16

In [ ]:

solution = solution_id_encoded
WINDOW_SIZE = 5
X1, y1 = df_to_X_y(solution, WINDOW_SIZE)
X1.shape, y1.shape

KeyError: ignored

In [ ]:
X_train1, y_train1 = X1[:60000], y1[:60000]
X_val1, y_val1 = X1[60000:65000], y1[60000:65000]
X_test1, y_test1 = X1[65000:], y1[65000:]
X_train1.shape, y_train1.shape, X_val1.shape, y_val1.shape, X_test1.shape, y_test1.shape

((57574, 5, 1, 177),
 (57574, 177),
 (0, 5, 1, 177),
 (0, 177),
 (0, 5, 1, 177),
 (0, 177))

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam

model1 = Sequential()
model1.add(InputLayer((5, 1)))
model1.add(LSTM(64))
model1.add(Dense(8, 'relu'))
model1.add(Dense(1, 'linear'))
model1.summary()

#model1 = Sequential()
#model1.add(LSTM(64, input_shape=(5, 1)))
#model1.add(Dense(1, activation='linear'))

#model1.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=0.001), metrics=[RootMeanSquaredError()])


Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_5 (LSTM)               (None, 64)                16896     
                                                                 
 dense_8 (Dense)             (None, 8)                 520       
                                                                 
 dense_9 (Dense)             (None, 1)                 9         
                                                                 
Total params: 17425 (68.07 KB)
Trainable params: 17425 (68.07 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
cp1 = ModelCheckpoint('model1/', save_best_only=True)
model1.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=0.0001), metrics=[RootMeanSquaredError()])

In [ ]:
print(np.isnan(X_train1).any())
print(np.isnan(y_train1).any())

False
False


In [ ]:
print(X_train1.shape)
print(y_train1.shape)

(57574, 5, 1)
(57574,)


In [ ]:
print(X_train1.dtype)

int64


In [ ]:
model1.fit(X_train1, y_train1, validation_data=(X_val1, y_val1), epochs=10, callbacks=[cp1])

Epoch 1/10


ValueError: ignored